# Simplex Method for Tableau Form

In this notebook there are the two implementation of Simplex Method for Tableau Form:
* Primal Simplex Method
* Dual Simplex Method

In addition, there is an implementatio of the **Two Phase Method**

In [1]:
import numpy as np

Function to perform the pivot overation on the tableau at position (t, h)

In [2]:
#parameters: tableau, index t of the row, index h of the column
def pivot_operation(tableau, t, h):
    #shape of the tableau
    m = tableau.shape[0]
    n = tableau.shape[1]
    pivot = tableau[t][h]
    #divide the pivot row by the pivot element
    for i in range(n):
        tableau[t][i] /= pivot
    #update all the other rows
    save = pivot
    for i in range(m):
        if i != t and tableau[i][h] != 0:
            save = tableau[i][h]
            for j in range(n):
                tableau[i][j] -= save*tableau[t][j]

## Primal Simplex Method

In [3]:
def primal_simplex_method(tableau, beta, two_phase=False, verbose=False):
    #print the starting tableau
    print('START TABLEAU:\n{}\n'.format(tableau))

    #number of rows in the tableau
    m = tableau.shape[0]
    #number of columns in the tableau
    n = tableau.shape[1]

    #final cases
    unbounded = False
    optimal = False

    while optimal == False and unbounded == False:
        #get the vector of costs
        costs = [tableau[0][c] for c in range(1, n)]
        #verify if all the costs are >= 0, thus the tableau is in optimal form
        if all(c >= 0 for c in costs):
            optimal = True
            break
        else:
            #index of the non basic variable
            h = 0
            #Find the first cost < 0 and choose a non basic variable
            for i, c in enumerate(tableau[0]):
                if i != 0 and c < 0:
                    h = i
                    #print('Variable of index {} will enter the basis'.format(h))
                    break
            #check if all the a[i, h] are < 0 so the problem is unbounded
            if all(tableau[i, h] < 0 for i in range(m)):
                unbounded = True
                break
            else:
                #choose the variable that will leave the basis
                min = 100000
                t = 1
                for i in range(1, m):
                    if tableau[i][h] > 0 and tableau[i][0] / tableau[i][h] < min:
                        min = tableau[i][0] / tableau[i][h]
                        t = i
                #print('Pivot operation on the cell: {}\n'.format((t, h)))

                #pivot operation
                pivot_operation(tableau, t, h)

                #update the vector beta, containing the indices of the basis variables
                beta[t-1] = h
        
        #Print the intermediate tableau
        if verbose:
            print('TABLEAU:\n{}\n'.format(tableau))

    #solution
    x = [0]*(n - 1)
    for i, b in enumerate(beta):
        x[b-1] = tableau[i+1][0]
    
    if two_phase:
        return tableau, beta

    if optimal:
        print('The tableau has an optimal solution x = ', x)
    elif unbounded:
        print('The tableau is unbounded')

### Dual Simplex Method

In [79]:
def dual_simplex_method(tableau, beta, verbose=False):
    #print the starting tableau
    print('START TABLEAU:\n{}\n'.format(tableau))

    #number of rows in the tableau
    m = tableau.shape[0]
    #number of columns in the tableau
    n = tableau.shape[1]

    #final cases
    infeasible = False
    optimal = False

    while optimal == False and infeasible == False:
        #get the vector of costs
        b_vector = [tableau[b][0] for b in range(1, m)]
        #verify if all the costs are >= 0, thus the tableau is in optimal form
        if all(b >= 0 for b in b_vector):
            optimal = True
            break
        else:
            #index of basic variable that will leave the basis
            t = 1
            #find the first negative variable with b < 0
            for i in range(1, m):
                if i != 0 and tableau[i][0] < 0:
                    t = i
                    break
            if all(tableau[t][i] >= 0 for i in range(1, n)):
                infeasible = True
                break
            else:
                #choose the variable that will enter the basis
                min = 100000
                h = 1
                for i in range(1, n):
                    if tableau[t][i] < 0 and tableau[0][i] / abs(tableau[t][i]) < min:
                        min = tableau[0][i] / abs(tableau[t][i])
                        h = i
                if verbose:
                    print('Pivot operation on the cell: {}\n'.format((t, h)))
                
                #pivot operation
                pivot_operation(tableau, t, h)

                #update the vector beta, containing the indices of the basis variables
                beta[t-1] = h
        
        #Print the intermediate tableau
        if verbose:
            print('TABLEAU:\n{}\n'.format(tableau))

    #solution
    x = [0]*(n - 1)
    for i, b in enumerate(beta):
        x[b-1] = tableau[i+1][0]
    if optimal:
        print('The tableau has an optimal solution x = ', x)
    elif infeasible:
        print('The tableau is infeasible')

## Two-Phase Method

Function to check if the tableau's columns can be combined to form the identity matrix $I$

In [4]:
def check_identity(tableau):
    #shape of the tableau
    m = tableau.shape[0]
    n = tableau.shape[1]
    #a column has shape of (m-1)x1, there are n-1 columns of variables and the matrix I is of shape (m-1)x(m-1)
    #all the columns of the identity matrix
    I = []
    for i in range(m-1):
        #create an empty column and the add the 1 in the i-th position
        col = [0.]*(m-1)
        col[i] = 1.
        I.append(col)

    #costs vector
    costs = [tableau[0][c] for c in range(n)]

    #check if there is the identity matrix in the input tableau
    for j in range(1, n):
        if not I:
            return True
        column = [tableau[i][j] for i in range(1, m)]
        if column in I and costs[j] == 0:
            I.remove(column)
            print('Removed column: ', column)
    
    if not I:
        return True
    else:
        return False

Phase One function, adding variables for the start basis

In [41]:
def phase_one(tableau):
    #shape of the tableau
    m = tableau.shape[0]
    n = tableau.shape[1]

    #counter for artificial variables
    counter = 0

    #add the new identity columns to the tableau
    for i in range(m-1):
        #create the new row with already the 0 as the cost
        arr = np.zeros((m, 1), dtype=float)
        arr[i+1] = 1.
        #add the new column to the tableau
        tableau = np.hstack((tableau, arr))
        #update counter for artificial variables
        counter += 1
    
    #calculate the new cost vector row w
    for j in range(n):
        tableau[0][j] = -sum(tableau[i][j] for i in range(1, m))

    print('At the end of Phase-One:', tableau)

    beta = [m+1, m+2]
    return tableau, beta, counter

Phase Two code

In [ ]:
#def phase_two(tableau)

Complete algorithm for the Two-Phase Method

In [48]:
def two_phase_method(tableau):
    #shape of the tableau
    m = tableau.shape[0]
    n = tableau.shape[1]
    #initial cost vector
    costs = [tableau[0][c] for c in range(1, n)]
    
    #run phase one
    tableau_one, beta_one, counter = phase_one(tableau)
    

    #solve with the primal simplex method
    tableau_int, beta = primal_simplex_method(tableau_one, beta_one, two_phase=True)

    print('After simplex method:\n', tableau_int)
    #find the rows of the basis column, before expressed in z
    new_z = np.zeros((tableau_int.shape[1]), dtype=float)
    for i, c in enumerate(costs):
        if (i+1) in beta and c != 0:
            #get the index of the interested row in the tableau
            row_index = beta.index(i+1)
            #add the row in the new cost vector z, after the coefficient of the basic variable has been removed
            row = tableau_int[row_index+1]
            row[i] = 0.
            #np.subtract(new_z, np.array(row))
            #TODO: modified the way of obtaining the new z after phase one
            for j in range(new_z.shape[0]):
                new_z[j] += c * row[j]
    #update the costs vector row of the tableau
    #print(new_z)
    for i in range(tableau_int.shape[1]):
        tableau_int[0][i] = new_z[i]
    print(tableau_int)

    #remove the columns of the artificial variables
    mod_tableau = tableau_int
    for i in range(counter):
        mod_tableau = np.delete(mod_tableau, mod_tableau.shape[1]-1, 1)
    print(mod_tableau)

    #apply the primal simplex method
    primal_simplex_method(mod_tableau, beta)


### Tests

#### 1: Primal Simplex Method
Test the Primal Simplex Method on the following tableau example

In [9]:
tableau = np.array([[0., -1., -1., 0., 0.], [24., 6., 4., 1., 0.], [6., 3., -2., 0., 1.]])
beta = np.array([1, 2])

#primal simple method only
primal_simplex_method(tableau, beta, verbose=True)

START TABLEAU:
[[ 0. -1. -1.  0.  0.]
 [24.  6.  4.  1.  0.]
 [ 6.  3. -2.  0.  1.]]

TABLEAU:
[[ 2.          0.         -1.66666667  0.          0.33333333]
 [12.          0.          8.          1.         -2.        ]
 [ 2.          1.         -0.66666667  0.          0.33333333]]

TABLEAU:
[[ 4.5         0.          0.          0.20833333 -0.08333333]
 [ 1.5         0.          1.          0.125      -0.25      ]
 [ 3.          1.          0.          0.08333333  0.16666667]]

TABLEAU:
[[ 6.    0.5   0.    0.25  0.  ]
 [ 6.    1.5   1.    0.25  0.  ]
 [18.    6.    0.    0.5   1.  ]]

The tableau has an optimal solution x =  [0, 6.0, 0, 18.0]


#### 2: Dual Simplex Method
Test the Dual Simplex Method on the following tableau example

In [15]:
tableau2 = np.array([[0., 0., 0., 0., 4., 3., 1., 0.], [-1., 0., 1., 0., -5., 1., 3., 0.], [-5., 1., 0., 0., -1., 0., 4., 0.], [-3., 0., 0., 1., 0., -1., 3., 0.], [-5., 0., 0., 0., 0., 2., -3., 1.]])
beta2 = np.array([2, 1, 3, 7])

#primal simple method only
dual_simplex_method(tableau2, beta2, verbose=True)

START TABLEAU:
[[ 0.  0.  0.  0.  4.  3.  1.  0.]
 [-1.  0.  1.  0. -5.  1.  3.  0.]
 [-5.  1.  0.  0. -1.  0.  4.  0.]
 [-3.  0.  0.  1.  0. -1.  3.  0.]
 [-5.  0.  0.  0.  0.  2. -3.  1.]]

Pivot operation on the cell: (1, 4)

TABLEAU:
[[-0.8  0.   0.8  0.   0.   3.8  3.4  0. ]
 [ 0.2 -0.  -0.2 -0.   1.  -0.2 -0.6 -0. ]
 [-4.8  1.  -0.2  0.   0.  -0.2  3.4  0. ]
 [-3.   0.   0.   1.   0.  -1.   3.   0. ]
 [-5.   0.   0.   0.   0.   2.  -3.   1. ]]

Pivot operation on the cell: (2, 2)

TABLEAU:
[[-20.   4.   0.   0.   0.   3.  17.   0.]
 [  5.  -1.   0.  -0.   1.   0.  -4.  -0.]
 [ 24.  -5.   1.  -0.  -0.   1. -17.  -0.]
 [ -3.   0.   0.   1.   0.  -1.   3.   0.]
 [ -5.   0.   0.   0.   0.   2.  -3.   1.]]

Pivot operation on the cell: (3, 5)

TABLEAU:
[[-29.   4.   0.   3.   0.   0.  26.   0.]
 [  5.  -1.   0.  -0.   1.   0.  -4.  -0.]
 [ 21.  -5.   1.   1.   0.   0. -14.   0.]
 [  3.  -0.  -0.  -1.  -0.   1.  -3.  -0.]
 [-11.   0.   0.   2.   0.   0.   3.   1.]]

The tableau is infe

#### 3: Two-Phase Method
Test the Two-Phase Method on the following tableau example

In [47]:
tableau3 = np.array([[-6, -5., -3., 4.], [1., 2., 1., -3.], [5., 3., 2., -1.]])

identity = check_identity(tableau3)
print(identity)

two_phase_method(tableau3)

False
At the end of Phase-One: [[-6. -5. -3.  4.  0.  0.]
 [ 1.  2.  1. -3.  1.  0.]
 [ 5.  3.  2. -1.  0.  1.]]
START TABLEAU:
[[-6. -5. -3.  4.  0.  0.]
 [ 1.  2.  1. -3.  1.  0.]
 [ 5.  3.  2. -1.  0.  1.]]

After simplex method:
 [[ 0.   0.   0.   0.   1.   1. ]
 [ 2.8  1.4  1.   0.  -0.2  0.6]
 [ 0.6 -0.2  0.   1.  -0.4  0.2]]
[[-6.  -0.8 -3.   4.  -1.  -1. ]
 [ 2.8  0.   1.   0.  -0.2  0.6]
 [ 0.6 -0.2  0.   1.  -0.4  0.2]]
[[-6.  -0.8 -3.   4. ]
 [ 2.8  0.   1.   0. ]
 [ 0.6 -0.2  0.   1. ]]
START TABLEAU:
[[-6.  -0.8 -3.   4. ]
 [ 2.8  0.   1.   0. ]
 [ 0.6 -0.2  0.   1. ]]



<ipython-input-2-2866cfeb9858>:9: RuntimeWarning: divide by zero encountered in double_scalars
  tableau[t][i] /= pivot
<ipython-input-2-2866cfeb9858>:9: RuntimeWarning: invalid value encountered in double_scalars
  tableau[t][i] /= pivot
<ipython-input-3-78ed7089cc1f>:39: RuntimeWarning: invalid value encountered in double_scalars
  if tableau[i][h] > 0 and tableau[i][0] / tableau[i][h] < min:


KeyboardInterrupt: 